In [1]:
"hello"

'hello'

In [2]:
# 처리량 측정 실험
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer
import json
import time
import random
from datetime import datetime

admin = AdminClient({"bootstrap.servers": "kafka:9092"})


# -----------------------------------------------------------------------------
# 토픽 생성 함수
# -----------------------------------------------------------------------------
def create_topic(name, num_partitions):
    """토픽 생성 (이미 존재하면 삭제 후 생성)"""
    # 기존 토픽 삭제 시도
    try:
        admin.delete_topics([name]).get(name).result()
        time.sleep(2)  # 삭제 완료 대기
    except:
        pass

    # 새 토픽 생성
    topic = NewTopic(topic=name, num_partitions=num_partitions, replication_factor=1)
    fs = admin.create_topics([topic])
    fs[name].result()
    time.sleep(1)
    print(f"토픽 '{name}' 생성 (파티션: {num_partitions})")


# -----------------------------------------------------------------------------
# 처리량 측정 함수
# -----------------------------------------------------------------------------
def measure_throughput(topic_name, num_messages):
    """메시지 전송 처리량 측정"""
    producer = Producer({"bootstrap.servers": "kafka:9092"})

    sent_count = 0

    def callback(err, msg):
        nonlocal sent_count
        if not err:
            sent_count += 1

    start_time = time.time()

    for i in range(num_messages):
        event = {
            "request_id": f"REQ_{i:06d}",
            "user_id": f"U{random.randint(1, 1000):04d}",
            "endpoint": f"/api/test",
            "timestamp": datetime.now().isoformat(),
        }

        producer.produce(
            topic=topic_name,
            value=json.dumps(event).encode("utf-8"),
            callback=callback,
        )

        # 주기적으로 flush
        if (i + 1) % 10000 == 0:
            producer.flush()

    producer.flush()
    elapsed = time.time() - start_time

    return sent_count, elapsed


# -----------------------------------------------------------------------------
# 실험 실행
# -----------------------------------------------------------------------------
NUM_MESSAGES = 50000

print("=" * 60)
print("파티션 수에 따른 처리량 비교")
print("=" * 60)

# 실험 1: 파티션 1개
create_topic("throughput-test-1p", 1)
count1, time1 = measure_throughput("throughput-test-1p", NUM_MESSAGES)
throughput1 = count1 / time1
print(f"  파티션 1개: {throughput1:,.0f} records/sec ({time1:.2f}초)")

# 실험 2: 파티션 4개
create_topic("throughput-test-4p", 4)
count4, time4 = measure_throughput("throughput-test-4p", NUM_MESSAGES)
throughput4 = count4 / time4
print(f"  파티션 4개: {throughput4:,.0f} records/sec ({time4:.2f}초)")

# 비교
print(f"\n결과:")
print(f"  처리량 향상: {(throughput4 / throughput1 - 1) * 100:.1f}%")

파티션 수에 따른 처리량 비교
토픽 'throughput-test-1p' 생성 (파티션: 1)
  파티션 1개: 154,224 records/sec (0.32초)
토픽 'throughput-test-4p' 생성 (파티션: 4)
  파티션 4개: 146,355 records/sec (0.34초)

결과:
  처리량 향상: -5.1%


In [3]:
# Producer: 파티션 4개 토픽에 100만 건 전송
from confluent_kafka import Producer
from confluent_kafka.admin import AdminClient, NewTopic
import json
import random
from datetime import datetime
import time

# -----------------------------------------------------------------------------
# 토픽 생성 (파티션 4개)
# -----------------------------------------------------------------------------
TOPIC = "api-events"

admin = AdminClient({"bootstrap.servers": "kafka:9092"})

# 기존 토픽 삭제 후 재생성
try:
    admin.delete_topics([TOPIC])[TOPIC].result()
    time.sleep(2)
    print(f"기존 토픽 '{TOPIC}' 삭제")
except:
    pass

topic = NewTopic(topic=TOPIC, num_partitions=4, replication_factor=1)
admin.create_topics([topic])[TOPIC].result()
print(f"토픽 '{TOPIC}' 생성 (파티션: 4개)\n")

# -----------------------------------------------------------------------------
# Producer 설정
# -----------------------------------------------------------------------------
config = {
    "bootstrap.servers": "kafka:9092",
    "client.id": "api-event-producer",
}

producer = Producer(config)

# -----------------------------------------------------------------------------
# API 이벤트 생성 함수
# -----------------------------------------------------------------------------
ENDPOINTS = ["/api/products", "/api/users", "/api/orders", "/api/payments", "/api/search"]
METHODS = ["GET", "POST", "PUT", "DELETE"]
STATUS_CODES = [200, 200, 200, 200, 201, 400, 404, 500]


def generate_api_event():
    return {
        "request_id": f"REQ_{random.randint(1, 999999):06d}",
        "user_id": f"U{random.randint(1, 1000):04d}",
        "endpoint": random.choice(ENDPOINTS),
        "method": random.choice(METHODS),
        "status_code": random.choice(STATUS_CODES),
        "response_time_ms": random.randint(10, 500),
        "timestamp": datetime.now().isoformat(),
    }


# -----------------------------------------------------------------------------
# Delivery Callback
# -----------------------------------------------------------------------------
sent_count = 0


def delivery_callback(err, msg):
    global sent_count
    if err:
        print(f"전송 실패: {err}")
    else:
        sent_count += 1


# -----------------------------------------------------------------------------
# 메시지 100만 건 전송
# -----------------------------------------------------------------------------
NUM_MESSAGES = 1000000

print(f"API 이벤트 {NUM_MESSAGES:,}건 전송 시작...")
start_time = time.time()

for i in range(NUM_MESSAGES):
    event = generate_api_event()

    producer.produce(
        topic=TOPIC,
        value=json.dumps(event).encode("utf-8"),
        callback=delivery_callback,
    )

    if (i + 1) % 100000 == 0:
        producer.flush()
        print(f"  {i + 1:,}건 전송 완료")

producer.flush()
elapsed = time.time() - start_time

print("\n전송 완료!")
print(f"  총 전송: {sent_count:,}건")
print(f"  소요 시간: {elapsed:.2f}초")
print(f"  처리량: {sent_count / elapsed:,.0f} records/sec")

기존 토픽 'api-events' 삭제
토픽 'api-events' 생성 (파티션: 4개)

API 이벤트 1,000,000건 전송 시작...
  100,000건 전송 완료
  200,000건 전송 완료
  300,000건 전송 완료
  400,000건 전송 완료
  500,000건 전송 완료
  600,000건 전송 완료
  700,000건 전송 완료
  800,000건 전송 완료
  900,000건 전송 완료
  1,000,000건 전송 완료

전송 완료!
  총 전송: 1,000,000건
  소요 시간: 10.51초
  처리량: 95,144 records/sec


In [4]:
# Consumer: 병렬 처리 (노트북 2개에서 동시 실행)
from confluent_kafka import Consumer
import json
import time

config = {
    "bootstrap.servers": "kafka:9092",
    "group.id": "parallel-consumer-group",  # 같은 그룹 ID!
    "auto.offset.reset": "earliest",
}

consumer = Consumer(config)
consumer.subscribe(["api-events"])

print("Consumer 시작...")
count = 0
start = None
last_msg_time = None

empty_count = 0
max_empty = 3  # 1초 x 3번 = 3초 대기 후 종료
first_message = True

try:
    while True:
        # 첫 메시지는 10초 대기, 이후 1초 대기
        timeout = 10.0 if first_message else 1.0
        msg = consumer.poll(timeout)

        if msg is None:
            empty_count += 1
            if first_message:
                print("10초 동안 메시지 없음. 종료합니다.")
                break
            if empty_count >= max_empty:
                print("더 이상 메시지 없음. 종료합니다.")
                break
            continue

        if msg.error():
            continue

        empty_count = 0

        # 첫 메시지 수신 시 시간 측정 시작
        if first_message:
            start = time.time()
            first_message = False

        last_msg_time = time.time()
        count += 1

        if count % 100000 == 0:
            print(f"  {count:,}건 처리중 (파티션 {msg.partition()})")

except KeyboardInterrupt:
    pass
finally:
    consumer.close()

# 결과 출력 (대기 시간 제외, 실제 처리 시간만 측정)
print(f"\n{'=' * 50}")
if count > 0 and start and last_msg_time:
    elapsed = last_msg_time - start
    print(f"총 수신: {count:,}건")
    print(f"소요 시간: {elapsed:.2f}초 (대기 시간 제외)")
    print(f"처리량: {count / elapsed:,.0f} records/sec")
else:
    print("수신된 메시지 없음")

Consumer 시작...
  100,000건 처리중 (파티션 0)
  200,000건 처리중 (파티션 0)
  300,000건 처리중 (파티션 1)
  400,000건 처리중 (파티션 0)
  500,000건 처리중 (파티션 0)
더 이상 메시지 없음. 종료합니다.

총 수신: 508,021건
소요 시간: 2.48초 (대기 시간 제외)
처리량: 204,678 records/sec
